In [132]:
!pip3 install pyperclip

  Created wheel for pyperclip: filename=pyperclip-1.8.0-py3-none-any.whl size=8691 sha256=b81395f6c53abcd74da57c26dd20aeeaf3d5486dd2bfc15b5b7b09172bc5d6df
  Stored in directory: /Users/leeminho/Library/Caches/pip/wheels/26/30/fe/92e2d4b1301ba74c07ea09c9e4c08f5bf12bae9c30319d74c5
Successfully built pyperclip


In [305]:
import re

In [146]:
import pandas as pd
import numpy as np

import requests # urllib 말고 requests 써주세요 (urllib은 구식입니다!)
from bs4 import BeautifulSoup as bs
import lxml
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import time
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


from collections import OrderedDict
from itertools import repeat
import pyperclip

In [147]:
import os
os.listdir('/Users/leeminho/Downloads')

['ITC Avant Garde Gothic LT Demi Regular.zip',
 '.Rhistory',
 '.DS_Store',
 'chromedriver_mac64.zip',
 '.localized',
 '.vscode 4',
 '.vscode 3',
 '.vscode 2',
 '.vscode 5',
 'BetterTouchTool.zip',
 '.vscode.zip',
 'Keka-1.1.26.dmg',
 '.vscode 10',
 'background.jpeg',
 '.vscode 11',
 'RStudio-1.2.5033.dmg',
 'vscode.zip',
 '.RData',
 '.ipynb_checkpoints',
 'chromedriver',
 '.vscode 9',
 '.vscode 7',
 '.vscode',
 '.vscode 6',
 '.vscode 8',
 'BetterTouchTool.app',
 'TT_comment.csv']

In [472]:
path = '/Users/leeminho/Downloads/chromedriver' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.x
driver = webdriver.Chrome(path)
driver.implicitly_wait(3) #혹은 time.sleep(3)

In [473]:
driver.get('https://cafe.naver.com/suhui?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10197921%26search.menuid=3853%26search.boardtype=L')
driver.implicitly_wait(3)

In [ ]:
driver.switch_to.frame('cafe_main')

In [ ]:
def get_url_list(link, start, end):
    url_list = []
    for i in range(start, end):
    if i == start:
        driver.get(link)
        driver.implicitly_wait(3)
        driver.switch_to.frame('cafe_main')
        soup = bs(driver.page_source, 'html.parser')
        for link in soup.find_all('a'):
            if 'href' in link.attrs:
                if 'ArticleRead' in link.attrs['href'].split('\n')[0]:
                    if 'specialmenutype' in link.attrs['href'].split('\n')[0] or 'commentFocus' in link.attrs['href'].split('\n')[0]:
                        continue
                    else:
                        url_list.append(link.attrs['href'].split('\n')[0])
    else:
        driver.get(link)
        driver.implicitly_wait(3)
        driver.switch_to.frame('cafe_main')
        soup = bs(driver.page_source, 'html.parser')
        for link in soup.find_all('a'):
            if 'href' in link.attrs:
                if 'ArticleRead' in link.attrs['href'].split('\n')[0]:
                    if 'specialmenutype' in link.attrs['href'].split('\n')[0] or 'commentFocus' in link.attrs['href'].split('\n')[0]:
                        continue
                    else:
                        url_list.append(link.attrs['href'].split('\n')[0])
    return url_list
    

In [ ]:
link = 'https://cafe.naver.com/suhui?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10197921%26search.menuid=3853%26search.boardtype=L'
get_url_list(link, 1, 71)

In [415]:
len(url_list)

1050

In [421]:
url_list = pd.DataFrame(url_list)
url_list.to_csv('수만휘_url_list.csv')

In [423]:
url_list = pd.read_csv('수만휘_url_list.csv')
url_list

,Unnamed: 0,0
0,0,/ArticleRead.nhn?clubid=10197921&page=1&menuid...
1,1,/ArticleRead.nhn?clubid=10197921&page=1&menuid...
2,2,/ArticleRead.nhn?clubid=10197921&page=1&menuid...
3,3,/ArticleRead.nhn?clubid=10197921&page=1&menuid...
4,4,/ArticleRead.nhn?clubid=10197921&page=1&menuid...
...,...,...
1045,1045,/ArticleRead.nhn?clubid=10197921&page=70&menui...
1046,1046,/ArticleRead.nhn?clubid=10197921&page=70&menui...
1047,1047,/ArticleRead.nhn?clubid=10197921&page=70&menui...
1048,1048,/ArticleRead.nhn?clubid=10197921&page=70&menui...


In [427]:
url_list = url_list['0'].to_list()
len(url_list)

1050

In [ ]:
def crawling(url_list):
    df = pd.DataFrame()
    for i in range(len(url_list)):
        driver.implicitly_wait(10)
        driver.get('https://cafe.naver.com' + url_list[i])
        time.sleep(2)
        driver.switch_to.frame('cafe_main')
        dict = {}
        
        #날짜 크롤링
        date = bs(driver.page_source, 'html.parser').find_all('span', {'class':'date'})
        driver.implicitly_wait(10)
        #페이지가 아직 안넘어갔을때
        if len(date) == 0:
            time.sleep(3)
            date = bs(driver.page_source, 'html.parser').find_all('span', {'class':'date'})
        date = date[0].text[:10]
        dict['date'] = date
        
        #제목 크롤링
        title = bs(driver.page_source, 'html.parser').find_all('h3', {'class':'title_text'})
        driver.implicitly_wait(10)
        title_text = re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9]',' ',title[0].text).strip()
        dict['word'] = [title_text]
        
        #게시물 내용, 댓글 크롤링
        content = bs(driver.page_source, 'html.parser').find_all('div', {'class':'ArticleContentBox'})
        list = content[0].find_all('p')[2:]
        new_list = []
        for j in range(len(list)):
            text = re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9]',' ',list[j].text).strip()
            if text != '':
                new_list.append(text)
        dict['word'] += new_list
        one_df = pd.DataFrame(dict)
        df = pd.concat([df, one_df])
        
        #crawling 도중 끊겼을 때 방지
        if i % 100 == 0:
            df.to_csv('수만휘crawling' + str(i) + '.csv')
        print(i, '번째 완료')
        
        return df
    

In [ ]:
crawling(url_list)

In [486]:
#함수만들기전
df = pd.DataFrame()
for i in range(701, 728):
    driver.implicitly_wait(10)
    driver.get('https://cafe.naver.com' + url_list[i])
    time.sleep(2)
    driver.switch_to.frame('cafe_main')
    dict = {}
    
    date = bs(driver.page_source, 'html.parser').find_all('span', {'class':'date'})
    driver.implicitly_wait(10)
    if len(date) == 0:
        time.sleep(3)
        date = bs(driver.page_source, 'html.parser').find_all('span', {'class':'date'})
    print(len(date))
    date = date[0].text[:10]
    dict['date'] = date
    
    title = bs(driver.page_source, 'html.parser').find_all('h3', {'class':'title_text'})
    driver.implicitly_wait(10)
    title_text = re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9]',' ',title[0].text).strip()
    dict['word'] = [title_text]
    
    content = bs(driver.page_source, 'html.parser').find_all('div', {'class':'ArticleContentBox'})
    driver.implicitly_wait(10)
    list = content[0].find_all('p')[2:]
    new_list = []
    for j in range(len(list)):
        text = re.sub('[^ㄱ-ㅣ가-힣a-zA-Z0-9]',' ',list[j].text).strip()
        if text != '':
            new_list.append(text)
    dict['word'] += new_list
    one_df = pd.DataFrame(dict)
    df = pd.concat([df, one_df])
    
    if i % 100 == 0:
        df.to_csv('수만휘crawling' + str(i) + '.csv')
    print(i, '번째 완료')
df.head()

1
701 번째 완료
1
702 번째 완료
1
703 번째 완료
1
704 번째 완료
1
705 번째 완료
1
706 번째 완료
1
707 번째 완료
1
708 번째 완료
1
709 번째 완료
1
710 번째 완료
1
711 번째 완료
1
712 번째 완료
1
713 번째 완료
1
714 번째 완료
1
715 번째 완료
1
716 번째 완료
1
717 번째 완료
1
718 번째 완료
1
719 번째 완료
1
720 번째 완료
1
721 번째 완료
1
722 번째 완료
1
723 번째 완료
1
724 번째 완료
1
725 번째 완료
1
726 번째 완료
1
727 번째 완료


,date,word
0,2019.10.16,조언좀요ㅠㅠ 과탐 현강 최수준 한종철
1,2019.10.16,1 음 유전은 시간 오래쓰면서 머리굴리는걸 꾸준히 연습해서 속도를 올려야해요 ...
0,2019.10.16,김승리쌤 현강
1,2019.10.16,승리쌤 현강 수업 결석했는데 보강 못가면 해당 주차 교재 못받나요 보강을 못 갈...
0,2019.10.14,시대인재에서 작년에 의대 mmi특강 들었던 분 계신가용


In [487]:
df.tail()

,date,word
4,2019.09.24,삭제된 댓글입니다
5,2019.09.24,감사합니다 포기하고 있었는데 물어 봐야겠네요
6,2019.09.24,혹시 전에 했던 중경수업도 2020 모의논술로 하셨나요
7,2019.09.24,수업은 1회 진행이 되었고요 중은 20년 모의논술로 진행 되었어요
0,2019.09.23,고3 수능대비 수수배학원 자체 제작 수학 가형 파이널 모의고사 발행 안내


In [579]:
#저장한 csv 파일 합쳐서 불러오기
df = pd.DataFrame()
for i in range(4):
    one_df = pd.read_csv('수만휘crawling(' + str(i+1) + ').csv')
    df = pd.concat([df, one_df])
df.head()

,Unnamed: 0,date,word
0,0,2020.06.01,학원가
1,1,2020.06.01,재수생이라 수강료가 많이 부담되서 그런데 수강료 괜찮은곳 아시는분
2,2,2020.06.01,가슴아픈 사연이지만 같은 모양의 가방이라도 10만원 짜리가 있고 5만원 짜리도...
3,3,2020.06.01,쪽지보내드렸습니다
4,0,2020.05.31,강안교육


In [580]:
df.tail()

,Unnamed: 0,date,word
1835,10,2019.03.30,킬캠은 왜여
1836,11,2019.03.30,미르임현강에서만 진행하는 킬캠 6 7회정도 추가로 더 해요
1837,0,2019.03.29,대치 시대 질문
1838,1,2019.03.29,아마 5월 초 쯤부터 현강을 다니려고 하는데요 조은정 차영진t 수업을 들으려고 ...
1839,2,2019.03.29,분당도 수업 듣는 사람 꽤 많을걸요


In [581]:
del df['Unnamed: 0']

In [582]:
df

,date,word
0,2020.06.01,학원가
1,2020.06.01,재수생이라 수강료가 많이 부담되서 그런데 수강료 괜찮은곳 아시는분
2,2020.06.01,가슴아픈 사연이지만 같은 모양의 가방이라도 10만원 짜리가 있고 5만원 짜리도...
3,2020.06.01,쪽지보내드렸습니다
4,2020.05.31,강안교육
...,...,...
1835,2019.03.30,킬캠은 왜여
1836,2019.03.30,미르임현강에서만 진행하는 킬캠 6 7회정도 추가로 더 해요
1837,2019.03.29,대치 시대 질문
1838,2019.03.29,아마 5월 초 쯤부터 현강을 다니려고 하는데요 조은정 차영진t 수업을 들으려고 ...


In [583]:
df.index = np.arange(0, len(df))
df

,date,word
0,2020.06.01,학원가
1,2020.06.01,재수생이라 수강료가 많이 부담되서 그런데 수강료 괜찮은곳 아시는분
2,2020.06.01,가슴아픈 사연이지만 같은 모양의 가방이라도 10만원 짜리가 있고 5만원 짜리도...
3,2020.06.01,쪽지보내드렸습니다
4,2020.05.31,강안교육
...,...,...
7005,2019.03.30,킬캠은 왜여
7006,2019.03.30,미르임현강에서만 진행하는 킬캠 6 7회정도 추가로 더 해요
7007,2019.03.29,대치 시대 질문
7008,2019.03.29,아마 5월 초 쯤부터 현강을 다니려고 하는데요 조은정 차영진t 수업을 들으려고 ...


In [584]:
df = df.iloc[:6980,:]

In [585]:
df = df.iloc[4:, :]
df.index = np.arange(0, len(df))
df

,date,word
0,2020.05.31,강안교육
1,2020.05.31,강안교육의 장단점이랑 추천 비추랑 가격좀 알려주실분 ㅠㅠ 기프티콘드려요
2,2020.05.31,쪽지주세요
3,2020.05.31,쪽지보냈습니다
4,2020.05.31,공부분위기는 진짜빡센데 화장실이 너무구려요 밥도 적게줘가지고 개인적으론 다시돌아가고...
...,...,...
6971,2019.04.02,국어는 어떤 선생님 수능 준비하기에 좋을까요 그리고 수학은 제가 나형 공부...
6972,2019.04.01,가형 80점정도에 적합한 현강
6973,2019.04.01,수학가형 80점정도 나와요대치동 현강중 적합한 커리나 쌤 추천부탁드립니다
6974,2019.04.01,현우진T 현강


In [586]:
df.to_csv('수만휘 크롤링(201904-202005).csv')

In [588]:
for i in range(len(df)):
    df['date'][i] = df['date'][i][:7]
df
        

,date,word
0,2020.05,강안교육
1,2020.05,강안교육의 장단점이랑 추천 비추랑 가격좀 알려주실분 ㅠㅠ 기프티콘드려요
2,2020.05,쪽지주세요
3,2020.05,쪽지보냈습니다
4,2020.05,공부분위기는 진짜빡센데 화장실이 너무구려요 밥도 적게줘가지고 개인적으론 다시돌아가고...
...,...,...
6971,2019.04,국어는 어떤 선생님 수능 준비하기에 좋을까요 그리고 수학은 제가 나형 공부...
6972,2019.04,가형 80점정도에 적합한 현강
6973,2019.04,수학가형 80점정도 나와요대치동 현강중 적합한 커리나 쌤 추천부탁드립니다
6974,2019.04,현우진T 현강


In [506]:
from konlpy.tag import Okt
okt = Okt()

In [523]:
#명사 빈도수 사전
dict = {}
for i in range(len(words)):
    nouns = okt.nouns(words[i])
    for j in range(len(nouns)):
        if nouns[j] in dict:
            dict[nouns[j]] += 1
        else:
            dict[nouns[j]] = 1
dict
    

{'공부': 174,
 '분위기': 66,
 '진짜': 167,
 '빡': 2,
 '화장실': 14,
 '구려': 2,
 '밥': 16,
 '개인': 79,
 '다시': 55,
 '저': 551,
 '고': 222,
 '현역': 33,
 '강안': 19,
 '윈터': 41,
 '스쿨': 19,
 '대화': 7,
 '친목': 6,
 '자체': 30,
 '본인': 55,
 '습관': 16,
 '추천': 410,
 '사실': 47,
 '수준': 93,
 '전국구': 1,
 '학생': 116,
 '대상': 12,
 '실제': 13,
 '시간': 219,
 '다른': 142,
 '메이저': 3,
 '강사': 83,
 '인강': 399,
 '친구': 74,
 '전체': 21,
 '학원': 614,
 '전반': 16,
 '시스템': 21,
 '쌤': 1180,
 '좀': 266,
 '경우': 72,
 '선배': 22,
 '크게': 16,
 '도움': 126,
 '시대': 227,
 '수반': 10,
 '합격': 40,
 '등록': 84,
 '뻔': 3,
 '김': 264,
 '승리': 265,
 '강': 1255,
 '메리트': 45,
 '작년': 128,
 '달리': 6,
 '올해': 85,
 '차이': 86,
 '것': 351,
 '나중': 26,
 '가면': 46,
 '현장': 82,
 '자료': 289,
 '반수': 9,
 '질문': 198,
 '제': 298,
 '학기': 11,
 '학교': 64,
 '주말': 34,
 '선택': 59,
 '갈수': 3,
 '보통': 57,
 '일주일': 19,
 '내내': 9,
 '통학': 7,
 '다가': 6,
 '널널': 3,
 '정보': 71,
 '아시': 66,
 '분': 377,
 '정예': 15,
 '집중': 35,
 '케어': 5,
 '재수': 136,
 '점': 136,
 '절박': 2,
 '자': 13,
 '스토리': 4,
 '해설': 40,
 '통합': 2,
 '사회': 10,
 '내신': 63,
 '한국사':

In [538]:
#명사빈도수 데이터프레임
nouns = pd.DataFrame(dict.items())
nouns.columns = ['word', 'counts']
nouns = nouns.sort_values(by = ['counts'], ascending = False)
nouns.index = np.arange(0, len(nouns))
nouns

,word,counts
0,강,1255
1,쌤,1180
2,수업,808
3,학원,614
4,저,551
...,...,...
4822,줄줄,1
4823,분야,1
4824,부디,1
4825,모레,1


In [617]:
#월별로 나누기
for i in range(len(np.unique(df['date'].values))):
    one_df = df[df.date == np.unique(df['date'].values)[i]]
    words = one_df['word'].values
    dict = {}
    for j in range(len(words)):
        nouns = okt.nouns(words[j])
        for k in range(len(nouns)):
            if nouns[k] in dict:
                dict[nouns[k]] += 1
            else:
                dict[nouns[k]] = 1
    nouns_df = pd.DataFrame(dict.items())
    nouns_df.columns = ['word', 'counts']
    nouns_df = nouns_df.sort_values(by = ['counts'], ascending = False)
    nouns_df.index = np.arange(0, len(nouns_df))
    print(nouns_df)
    if i <= 8:
        nouns_df.to_csv('월별 수만휘 크롤링 2019' + str(i+4) +'.csv')
    else:
        nouns_df.to_csv('월별 수만휘 크롤링 2020' + str(i-8) + '.csv')
    

    word  counts
0      강      67
1      쌤      40
2     수업      29
3    선생님      28
4     인강      24
..   ...     ...
547   일치       1
548   매번       1
549   제출       1
550   종이       1
551   가용       1

[552 rows x 2 columns]
    word  counts
0      강      45
1      쌤      29
2     수업      25
3     승리      22
4      요      21
..   ...     ...
641    듯       1
642   도중       1
643   출석       1
644   다차       1
645   서나       1

[646 rows x 2 columns]
    word  counts
0      강      64
1     수업      51
2      쌤      47
3     대기      27
4      분      24
..   ...     ...
853  금요일       1
854    노       1
855   강쌤       1
856   승홍       1
857  십만원       1

[858 rows x 2 columns]
    word  counts
0      강      87
1     수업      57
2      쌤      55
3     대기      46
4     학원      37
..   ...     ...
944   기준       1
945   카페       1
946  시간제       1
947   안해       1
948   낭찡       1

[949 rows x 2 columns]
    word  counts
0      강      33
1      저      23
2      쌤      21
3      분      19
4  